In [1]:
import numpy as np, pandas as pd

#get the submission file from GitHub
url='https://raw.githubusercontent.com/TRACK2-DATA-SCIENCE-2021/Track2_Challenges_Data/main/Challenge3/data/day2data.csv'
data=pd.read_csv(url, header=1)
sub_cols=['Bed','Ventilator','Remdesivir','Dexamethasone','Plasma','Casirivimab','Supplemental oxygen','Chloroquine','Total']
submission=pd.concat([data.iloc[:,:-1],pd.DataFrame(data=0, index=data.index, columns=sub_cols)], axis=1)
submission

,ID,Health,Rate of health decline,Bed,Ventilator,Remdesivir,Dexamethasone,Plasma,Casirivimab,Supplemental oxygen,Chloroquine,Total
0,21,54,-1,0,0,0,0,0,0,0,0,0
1,22,23,-20,0,0,0,0,0,0,0,0,0
2,23,8,-40,0,0,0,0,0,0,0,0,0
3,24,45,-41,0,0,0,0,0,0,0,0,0
4,25,100,-44,0,0,0,0,0,0,0,0,0
5,26,87,-45,0,0,0,0,0,0,0,0,0
6,27,65,-50,0,0,0,0,0,0,0,0,0
7,28,79,-60,0,0,0,0,0,0,0,0,0
8,29,56,-65,0,0,0,0,0,0,0,0,0
9,30,70,-71,0,0,0,0,0,0,0,0,0


In [2]:
import numpy as np

#Vector for quick addition of treatment
treatment_vec=np.array([0,1,1,5,30, 30,25,15,15,20,10])

#Sort sailors based on their predicted health at the end of day 2 with no treatment
day2health=submission['Health']+2*submission['Rate of health decline']
sickest_sailors=day2health.values.argsort(kind='mergesort')

#Put the thirty sickest (or all) sailors in beds
if submission.shape[0]<30:
  submission['Bed']=1
else:
  submission.loc[sickest_sailors[:30], 'Bed']=1

#Give the ten sickest (or all) sailors  ventilators 
if submission.shape[0]<10:
  submission['Ventilator']=1
else:
  submission.loc[sickest_sailors[:10], 'Ventilator']=1
  #Give the next ten sickest (or all) sailors sumplemental oxygen
  if submission.shape[0]<20:
    submission.loc[10:submission.shape[0], 'Supplemental oxygen']=1
  else:
    submission.loc[sickest_sailors[10:20], 'Supplemental oxygen']=1

#If health is greater than 90, the bonus points are worth more than health
submission.loc[lambda df:df['Health']+df['Rate of health decline']>90,
               ['Ventilator', 'Supplemental oxygen']]=0

#Recalculate total
perfect_fn=lambda x:100 if x>100 else x
submission['Total']=np.matmul(submission.values[:,:-1], treatment_vec)
submission['Total']=submission['Total'].apply(perfect_fn)
submission

,ID,Health,Rate of health decline,Bed,Ventilator,Remdesivir,Dexamethasone,Plasma,Casirivimab,Supplemental oxygen,Chloroquine,Total
0,21,54,-1,1,0,0,0,0,0,0,0,58
1,22,23,-20,1,0,0,0,0,0,0,0,8
2,23,8,-40,1,0,0,0,0,0,1,0,-7
3,24,45,-41,1,0,0,0,0,0,0,0,9
4,25,100,-44,1,0,0,0,0,0,0,0,61
5,26,87,-45,1,0,0,0,0,0,0,0,47
6,27,65,-50,1,0,0,0,0,0,0,0,20
7,28,79,-60,1,0,0,0,0,0,0,0,24
8,29,56,-65,1,0,0,0,0,0,1,0,16
9,30,70,-71,1,0,0,0,0,0,1,0,24


In [3]:
#Set a goal health for each sailor that they will survive until Day 3
#If health decline is -50 or less, assume they'll need a ventilator or supplemental ox
goal_vec=pd.Series(1, index=submission.index)

In [6]:
#Apply Remdesivir to sailors who need it to reach goal health and recalculate health
need_Rem=np.where(submission['Total'].values<goal_vec-26)[-1]
submission.loc[need_Rem,'Remdesivir']=1
submission['Total']=np.matmul(submission.values[:,:-1], treatment_vec)
submission['Total']=submission['Total'].apply(perfect_fn)

#Likewise for Dexamethasone
need_Dex=np.where(submission['Total'].values<goal_vec-21)[-1]
submission.loc[need_Dex,'Dexamethasone']=1
submission['Total']=np.matmul(submission.values[:,:-1], treatment_vec)
submission['Total']=submission['Total'].apply(perfect_fn)

#Use Plasma and Chloroquine for those just healthy enough to not need dexamethasone
need_Plasma_Chloro=np.where(submission['Total'].values<goal_vec-16)[-1]
submission.loc[need_Plasma_Chloro,['Plasma', 'Chloroquine']]=1
submission['Total']=np.matmul(submission.values[:,:-1], treatment_vec)
submission['Total']=submission['Total'].apply(perfect_fn)

#Next option is need_Casirivimab
need_Casirivimab=np.where(submission['Total'].values<goal_vec-11)[-1]
submission.loc[need_Casirivimab,'Casirivimab']=1
submission['Total']=np.matmul(submission.values[:,:-1], treatment_vec)
submission['Total']=submission['Total'].apply(perfect_fn)

#And lastly just plasma
need_Chloro=np.where(submission['Total'].values<goal_vec)[-1]
submission.loc[need_Chloro,'Chloroquine']=1
submission['Total']=np.matmul(submission.values[:,:-1], treatment_vec)
submission['Total']=submission['Total'].apply(perfect_fn)

submission

,ID,Health,Rate of health decline,Bed,Ventilator,Remdesivir,Dexamethasone,Plasma,Casirivimab,Supplemental oxygen,Chloroquine,Total
0,21,54,-1,1,0,0,0,0,0,0,0,58
1,22,23,-20,1,0,0,0,0,0,0,0,8
2,23,8,-40,1,0,0,0,0,0,1,1,3
3,24,45,-41,1,0,0,0,0,0,0,0,9
4,25,100,-44,1,0,0,0,0,0,0,0,61
5,26,87,-45,1,0,0,0,0,0,0,0,47
6,27,65,-50,1,0,0,0,0,0,0,0,20
7,28,79,-60,1,0,0,0,0,0,0,0,24
8,29,56,-65,1,0,0,0,0,0,1,0,16
9,30,70,-71,1,0,0,0,0,0,1,0,24


In [7]:
submission.to_csv('Challenge3_Day2Submission.csv', index=False)

In [8]:
#Check to make sure we haven't run out of anything
#It looks like we're all good
submission.sum(axis=0)

ID                         918
Health                    1722
Rate of health decline   -1943
Bed                         27
Ventilator                  10
Remdesivir                   3
Dexamethasone                0
Plasma                       2
Casirivimab                  1
Supplemental oxygen         10
Chloroquine                  6
Total                      609
dtype: int64

In [9]:
for x in range(20):
  print(submission.iloc[x,-1],goal_vec[x])

58 1
8 1
3 1
9 1
61 1
47 1
20 1
24 1
16 1
24 1
43 1
49 1
16 1
19 1
10 1
2 1
10 1
24 1
7 1
29 1
